In [1]:
from cesnet_datazoo.datasets import CESNET_QUIC22, CESNET_TLS22
from cesnet_datazoo.config import DatasetConfig
from networkx.drawing import to_latex

dataset = CESNET_QUIC22(data_root="data/CESNET_QUIC22/", size="XS", silent=True)

dataset_config = DatasetConfig(
    dataset=dataset,
    train_period_name="W-2022-44",
    train_size=100_000,
    use_packet_histograms=True,
    return_other_fields=True,
)
dataset.set_dataset_config_and_initialize(dataset_config)

data_df = dataset.get_train_df()

data_df["APP_NAME"] = None

# get the app number from 'app' column and translate it using the 'app_labels' dictionary
data_df["APP_NAME"] = data_df["APP"].apply(lambda x: dataset._tables_app_enum.get(x, "Unknown"))

categories_enum = {category: i for i, category in dataset._tables_cat_enum.items()}
app_to_categories = dataset.class_info.categories_mapping
app_enum = {i: app for i, app in dataset._tables_app_enum.items()}
app_to_categories.update({
    "bongacams": "Streaming media",
    "gothbb": "E-commerce",
    "vkontakte": "Social",
    "poe-ninja": "Games",
    "unpkg": "Streaming media",
    "easylist": "Other services and APIs",
    "default-background": "default",
    "uber": "Other services and APIs",
})

data_df["CATEGORY"] = data_df["APP"].apply(lambda x: app_to_categories[app_enum[x]])

columns = ["PACKETS", "PACKETS_REV", "BYTES", "BYTES_REV", "DURATION", "PPI_LEN", "PPI_ROUNDTRIPS", "PPI_DURATION"]

# Whole dataset

In [5]:
data_df[columns].describe(percentiles=[0.05, 0.5, 0.75, 0.95]).round(2)

,PACKETS,PACKETS_REV,BYTES,BYTES_REV,DURATION,PPI_LEN,PPI_ROUNDTRIPS,PPI_DURATION
count,100000.00,100000.00,1.000000e+05,1.000000e+05,100000.00,100000.00,100000.00,100000.00
mean,44.65,150.65,1.639926e+04,1.692565e+05,9.47,23.14,4.55,3.25
std,626.01,2088.69,6.881355e+05,2.636015e+06,29.61,7.27,1.88,12.31
min,1.00,1.00,1.228000e+03,5.900000e+01,0.00,2.00,0.00,0.00
5%,5.00,5.00,1.614000e+03,2.345000e+03,0.04,10.00,2.00,0.03
50%,12.00,13.00,4.178500e+03,5.257000e+03,0.22,25.00,4.00,0.16
75%,22.00,24.00,7.168000e+03,1.051200e+04,4.98,30.00,6.00,0.58
95%,113.05,257.00,3.524730e+04,2.497760e+05,47.37,30.00,8.00,19.47
max,162848.00,489305.00,2.060549e+08,6.242670e+08,356.12,30.00,15.00,291.12


# Easybrain

In [6]:
data_df[columns][data_df["APP_NAME"] == "easybrain"].describe(percentiles=[0.05, 0.5, 0.75, 0.95]).round(2)

,PACKETS,PACKETS_REV,BYTES,BYTES_REV,DURATION,PPI_LEN,PPI_ROUNDTRIPS,PPI_DURATION
count,4924.00,4924.00,4924.00,4924.00,4924.00,4924.00,4924.00,4924.00
mean,19.50,43.18,3811.88,42155.79,4.38,20.00,3.72,2.55
std,145.71,498.89,10475.12,610825.56,12.27,6.08,1.51,7.72
min,1.00,2.00,1228.00,1910.00,0.02,4.00,1.00,0.02
5%,4.00,7.00,1886.15,2963.00,0.06,11.00,2.00,0.05
50%,8.00,9.00,2532.00,3206.00,0.09,17.00,3.00,0.09
75%,12.00,13.00,3389.25,4042.00,0.31,25.00,4.00,0.14
95%,58.00,101.70,7232.50,101176.55,27.99,30.00,7.00,20.04
max,9447.00,22663.00,617954.00,27725098.00,150.30,30.00,11.00,90.15


# Gamedock

In [7]:
data_df[columns][data_df["APP_NAME"] == "gamedock"].describe(percentiles=[0.05, 0.5, 0.75, 0.95]).round(2)

,PACKETS,PACKETS_REV,BYTES,BYTES_REV,DURATION,PPI_LEN,PPI_ROUNDTRIPS,PPI_DURATION
count,283.00,283.00,2.830000e+02,283.00,283.00,283.00,283.00,283.00
mean,1525.14,276.75,1.913596e+06,37168.46,7.88,27.48,5.50,1.62
std,10089.01,1207.42,1.278219e+07,111053.34,15.95,4.90,1.44,4.67
min,1.00,3.00,1.228000e+03,3369.00,0.02,4.00,1.00,0.02
5%,6.10,10.00,3.844400e+03,3862.40,0.20,19.00,4.00,0.11
50%,19.00,19.00,6.832000e+03,9935.00,1.42,30.00,6.00,0.49
75%,148.00,96.50,8.168300e+04,22042.50,6.96,30.00,7.00,0.78
95%,4730.60,1087.80,5.914807e+06,133064.50,33.48,30.00,8.00,6.04
max,162848.00,16729.00,2.060549e+08,1001368.00,137.42,30.00,10.00,35.68


# Games category

In [8]:
data_df[columns][data_df["CATEGORY"] == "Games"].describe(percentiles=[0.05, 0.5, 0.75, 0.95]).round(2)

,PACKETS,PACKETS_REV,BYTES,BYTES_REV,DURATION,PPI_LEN,PPI_ROUNDTRIPS,PPI_DURATION
count,10261.00,10261.00,1.026100e+04,10261.00,10261.00,10261.00,10261.00,10261.00
mean,63.51,59.06,5.683323e+04,55672.07,3.98,19.66,3.66,2.01
std,1696.87,527.43,2.142102e+06,611714.81,11.06,7.62,1.62,6.49
min,1.00,2.00,1.228000e+03,1588.00,0.02,4.00,0.00,0.00
5%,4.00,4.00,1.610000e+03,2353.00,0.04,9.00,2.00,0.04
50%,8.00,9.00,2.591000e+03,3426.00,0.10,17.00,3.00,0.09
75%,15.00,17.00,3.899000e+03,8058.00,0.70,30.00,4.00,0.22
95%,77.00,192.00,9.321000e+03,203995.00,25.71,30.00,7.00,15.07
max,162848.00,22663.00,2.060549e+08,27725098.00,175.16,30.00,11.00,90.15
